# Programming in lower level



* Sometimes a control engineer may want to make computers to give commands to or read measurements from machines.



* Understanding how computers work *under the hood* would be helpful in such situations.



## Chinese Room



* Prof. John Searle of UC Berkeley published in 1980.



[![Chinese Room](https://cdn-images-1.medium.com/max/1600/1*ve1L0BoTEEChNZMwn7_zig.jpeg)](https://medium.com/@transphilosophr/searles-chinese-room-thought-experiment-a-twist-c7eb28f65e6c)



* Assume a person inside of a closed room with slots for questions and answers.



* From outside, a person would (in)put a letter with a question in an unfamiliar foreign language into the room.



* The person inside of the room has a very big book with answers to all questions of that foreign language.



* The person inside of the room would lookup the answer from the book and (out)put another letter in the foreign language with the answer out of the room.



* Now, do people outside of the room can tell if the person inside knows the foreign language?



### Analogy



* Let's compare this room with a computer.



| thought experiment | computer |
|:------:|:------:|
| room | CPU |
| person inside of the room | CPU electronics |
| book in the room | CPU instruction set |
| people outside of the room | memory |
| letter coming in | machine language instruction |
| letter going out | result of the machine language instruction |
| frequency of letter | clock speed |



## Classes of computer memories



| memory | physical location | amount | speed | purpose |
|:-----:|:-----:|:-----:|:-----:|:-----:|
| registers | CPU | very limited | fastest | operation, addressing, I/O |
| cache | In or close to CPU | not much | faster | accelerating information fetching |
| main memory | motherboard RAM | GBs nowadays | fast | store code and data |
| stack | within RAM | limited | RAM | store function call information |
| [heap](https://en.wikipedia.org/wiki/Memory_management#HEAP) | within RAM | less limited | RAM | allocate memory from here |
| SSD | SSD | TBs? | [Flash memory](https://en.wikipedia.org/wiki/Flash_memory) speed | preserve code and data  |
| HDD | HDD | TBs | Magnetic disk speed | preserve code and data  |
| Memory mapped I/O | ??? | limited | ??? | communicate with a device  |



* Following [figure](https://thenextweb.com/shareables/2011/12/26/this-is-what-a-5mb-hard-drive-looked-like-is-1956-required-a-forklift/) shows an early [hard disk drive](https://en.wikipedia.org/wiki/IBM_305_RAMAC) of 5 Megabytes in 1950s.



![IBM HDD 5MB 1956](https://cdn0.tnwcdn.com/wp-content/blogs.dir/1/files/2011/12/Screen-Shot-2011-12-26-at-18.38.18.png)



* Among these memories, cache is usually not accessable from the program.



* Disk drives, programs can access through file systems.



* Main memory (+ stack and heap) is *addressable*; thus we can use *pointer*s to access.



* How to access registers depends on the compilers.



* Memory mapped IO is addressable; thus we can use pointers to access.  However, if output, the computer would send what we write here to a device, possibly an external one.

